NOTE: 

This notebook was run with both the pd.get_dummies drop first parameter set to True and False. I saved the False one.

Observations on the differences:

I got higher scores returned from the RNN and LSTM models with this parameter set to False. 

Additional work may need to be done if I wish to continue using that deprecated method, predict_classes to run a classification report. 



## Keras and TensorFlow

In this assignment, we will learn about Keras and TensorFlow. We will create a neural network and measure the model's performance.

In [1]:
import numpy as np
import pandas as pd

In [2]:
cancer = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/breast_cancer.csv')

In [3]:
cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


We'll start with some data cleaning. First remove the id column since we have no use for it. Then check for missing data and remove all rows containing missing data.

In [4]:
# Answer below:
cancer = cancer.drop(columns='id').dropna()

The diagnosis columnn is our target variable. How many possible values are in this variable?

In [5]:
# Answer below:
cancer.diagnosis.nunique()

2

Create one or multiple dummy variables for this column and store this encoded data in a numpy array called `target`

In [6]:
# Answer below:
target = pd.get_dummies(cancer.diagnosis,drop_first=False)
target

,B,M
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
564,0,1
565,0,1
566,0,1
567,0,1


When looking at the columm names, you may have noticed that for each attribute we have a mean of the attribute, a standard error, and a worst value. Let's create a list of column names for only the means columns. Use this list to create a new dataframe containing only the means columns.

In [7]:
# Answer below:
means = [s for s in cancer.columns.values if "mean" in s]
df = cancer[means]

Split the data into train and test samples. The test sample should contain 20% of the data.

In [8]:
from sklearn.model_selection import train_test_split

X = df
y = target
# Answer below
#Split the data.
SIZE = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SIZE)
print('There are {:d} training samples and {:d} test samples'.format(X_train.shape[0], X_test.shape[0]))


There are 455 training samples and 114 test samples


Now we'll use the minmax scaler to ensure that all columns have values between zero and 1. Apply this transformations for all columns to be used in the prediction.

In [9]:
# Answer below:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()

Xm_train = mm.fit_transform(X_train)
Xm_test = mm.transform(X_test)



Now we can proceed with our model. In the cell below, import the `Sequential`, and `Dense` functions.

In [10]:
# Answer below:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

create a model that takes the training data and predicts the target variable. The model will have 3 layers. The input layer should be of unit size 12 and the hidden layer will be of size 10. The output layer size will be determined by the size of the target variable. Use the sigmoid activation function for the last layer

In [11]:
Xm_train.shape

(455, 10)

In [12]:
y_train.shape

(455, 2)

In [13]:
# Answer below
import keras
from keras import layers

units = 12
output_size = y_train.shape[1]
input_dim = Xm_train.shape[1]

model = Sequential()
# first dense
model.add(Dense(12, input_dim=input_dim, activation='relu'))
# model.add(Dense(1028, input_shape=(784,), activation='relu'))
# second dense
model.add(Dense(10, activation='relu'))
# last layer/output layer
model.add(Dense(output_size, activation='sigmoid'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                132       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 284
Trainable params: 284
Non-trainable params: 0
_________________________________________________________________


Compile and fit the model. Use the adam optimizer and the binary crossentropy loss function. Train the model for 200 epochs with a batch size of 80. Make sure touse the test data into the model as well.

In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])

In [16]:
# Answer below:

history =  model.fit(Xm_train, y_train, validation_data=(Xm_test, y_test), batch_size=80, epochs=200, verbose=1)

Epoch 1/200
6/6 [==============================] - 1s 84ms/step - loss: 0.6939 - accuracy: 0.3673 - val_loss: 0.6943 - val_accuracy: 0.3421
Epoch 2/200
6/6 [==============================] - 0s 8ms/step - loss: 0.6895 - accuracy: 0.3574 - val_loss: 0.6886 - val_accuracy: 0.3596
Epoch 3/200
6/6 [==============================] - 0s 9ms/step - loss: 0.6798 - accuracy: 0.4033 - val_loss: 0.6830 - val_accuracy: 0.4211
Epoch 4/200
6/6 [==============================] - 0s 32ms/step - loss: 0.6787 - accuracy: 0.4396 - val_loss: 0.6772 - val_accuracy: 0.4825
Epoch 5/200
6/6 [==============================] - 0s 9ms/step - loss: 0.6706 - accuracy: 0.5521 - val_loss: 0.6715 - val_accuracy: 0.5965
Epoch 6/200
6/6 [==============================] - 0s 9ms/step - loss: 0.6667 - accuracy: 0.6112 - val_loss: 0.6658 - val_accuracy: 0.7105
Epoch 7/200
6/6 [==============================] - 0s 8ms/step - loss: 0.6596 - accuracy: 0.7263 - val_loss: 0.6601 - val_accuracy: 0.7632
Epoch 8/200
6/6 [========

In [17]:
y_pred_pc = model.predict_classes(Xm_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [18]:
y_pred_np = (model.predict(Xm_test) > 0.5).astype("int32")

Report the accuracy of the test sample.

In [19]:
# Answer below:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_np))


              precision    recall  f1-score   support

           0       0.99      0.95      0.97        75
           1       0.90      0.97      0.94        39

   micro avg       0.96      0.96      0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114
 samples avg       0.96      0.96      0.96       114



In [20]:
print(classification_report(y_test, y_pred_pc))

ValueError: ignored

I'm learning how to build an RNN model as a part of a challenge that was given to me in which I have two days to complete a project that uses RNN and CNN to classify a bunch of images. 

I've decided to use this lesson as an example to help me out. 

In [21]:
# This is bad code - the data will continually provide an error message indicating that the shape of the data provided is not what's expected. 
# It errors out after trying to fit the model. 
import keras
from keras import layers

units = 12
output_size = y_train.shape[1]
input_dim = Xm_train.shape[1]

#sequential model
model = keras.models.Sequential([
  #Layer one with 12 units.
  #keras.layers.RNN(keras.layers.LSTMCell(units, input_dim=input_dim)),
  keras.layers.LSTM(units, input_dim=10, activation='relu'),
  keras.layers.Dense(output_size, activation='sigmoid')
])

In [22]:
from tensorflow.keras.layers import LSTM
model = Sequential()
# first dense
# output embedding dimension of size 10. This is the magic code that'll create the model. 
model.add(layers.Embedding(input_dim=10, output_dim=1))
#model.add(Dense(12, input_dim=input_dim,  activation='relu'))
# model.add(Dense(1028, input_shape=(784,), activation='relu'))
# second dense
model.add(LSTM(10))
# last layer/output layer
model.add(Dense(output_size, activation='sigmoid'))

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])

In [24]:
# Answer below:

history =  model.fit(Xm_train, y_train, validation_data=(Xm_test, y_test), batch_size=80, epochs=200, verbose=2)

Epoch 1/200
6/6 - 2s - loss: 0.6920 - accuracy: 0.6198 - val_loss: 0.6894 - val_accuracy: 0.6579
Epoch 2/200
6/6 - 0s - loss: 0.6893 - accuracy: 0.6198 - val_loss: 0.6857 - val_accuracy: 0.6579
Epoch 3/200
6/6 - 0s - loss: 0.6866 - accuracy: 0.6198 - val_loss: 0.6819 - val_accuracy: 0.6579
Epoch 4/200
6/6 - 0s - loss: 0.6837 - accuracy: 0.6198 - val_loss: 0.6778 - val_accuracy: 0.6579
Epoch 5/200
6/6 - 0s - loss: 0.6807 - accuracy: 0.6198 - val_loss: 0.6734 - val_accuracy: 0.6579
Epoch 6/200
6/6 - 0s - loss: 0.6773 - accuracy: 0.6198 - val_loss: 0.6686 - val_accuracy: 0.6579
Epoch 7/200
6/6 - 0s - loss: 0.6745 - accuracy: 0.6198 - val_loss: 0.6632 - val_accuracy: 0.6579
Epoch 8/200
6/6 - 0s - loss: 0.6708 - accuracy: 0.6198 - val_loss: 0.6586 - val_accuracy: 0.6579
Epoch 9/200
6/6 - 0s - loss: 0.6681 - accuracy: 0.6198 - val_loss: 0.6540 - val_accuracy: 0.6579
Epoch 10/200
6/6 - 0s - loss: 0.6667 - accuracy: 0.6198 - val_loss: 0.6498 - val_accuracy: 0.6579
Epoch 11/200
6/6 - 0s - loss:

In [25]:
y_pred_np = (model.predict(Xm_test) > 0.5).astype("int32")

In [26]:
# Answer below:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_np))


              precision    recall  f1-score   support

           0       0.66      1.00      0.80        75
           1       1.00      0.03      0.05        39

   micro avg       0.67      0.67      0.67       114
   macro avg       0.83      0.51      0.42       114
weighted avg       0.78      0.67      0.54       114
 samples avg       0.67      0.67      0.67       114



In [27]:
units = 12
output_size = y_train.shape[1]
input_dim = Xm_train.shape[1]

#sequential model
model = keras.models.Sequential([ 
  keras.layers.Embedding(input_dim=10, output_dim=1),
  #Layer two with 12 units.
  keras.layers.RNN(keras.layers.LSTMCell(units, input_dim=input_dim)),
  #Layer one with 12 units.
  #keras.layers.LSTM(units, input_dim=10, activation='relu'),
  keras.layers.Dense(output_size, activation='sigmoid')
])

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])
# Answer below:

history =  model.fit(Xm_train, y_train, validation_data=(Xm_test, y_test), batch_size=80, epochs=200, verbose=2)


Epoch 1/200
6/6 - 2s - loss: 0.6918 - accuracy: 0.5846 - val_loss: 0.6895 - val_accuracy: 0.6579
Epoch 2/200
6/6 - 0s - loss: 0.6894 - accuracy: 0.6198 - val_loss: 0.6856 - val_accuracy: 0.6579
Epoch 3/200
6/6 - 0s - loss: 0.6864 - accuracy: 0.6198 - val_loss: 0.6815 - val_accuracy: 0.6579
Epoch 4/200
6/6 - 0s - loss: 0.6833 - accuracy: 0.6198 - val_loss: 0.6770 - val_accuracy: 0.6579
Epoch 5/200
6/6 - 0s - loss: 0.6799 - accuracy: 0.6198 - val_loss: 0.6720 - val_accuracy: 0.6579
Epoch 6/200
6/6 - 0s - loss: 0.6763 - accuracy: 0.6198 - val_loss: 0.6663 - val_accuracy: 0.6579
Epoch 7/200
6/6 - 0s - loss: 0.6729 - accuracy: 0.6198 - val_loss: 0.6597 - val_accuracy: 0.6579
Epoch 8/200
6/6 - 0s - loss: 0.6694 - accuracy: 0.6198 - val_loss: 0.6538 - val_accuracy: 0.6579
Epoch 9/200
6/6 - 0s - loss: 0.6662 - accuracy: 0.6198 - val_loss: 0.6490 - val_accuracy: 0.6579
Epoch 10/200
6/6 - 0s - loss: 0.6649 - accuracy: 0.6198 - val_loss: 0.6461 - val_accuracy: 0.6579
Epoch 11/200
6/6 - 0s - loss:

In [29]:
y_pred_np = (model.predict(Xm_test) > 0.5).astype("int32")

In [30]:
print(classification_report(y_test, y_pred_np))


              precision    recall  f1-score   support

           0       0.66      1.00      0.80        75
           1       1.00      0.03      0.05        39

   micro avg       0.67      0.67      0.67       114
   macro avg       0.83      0.51      0.42       114
weighted avg       0.78      0.67      0.54       114
 samples avg       0.67      0.67      0.67       114

